# AugmentedGPT Playground

Please run `poetry install` under the project root directory. This will install all the necessary dependencies for you.

## Imports and set API key

In [1]:
from augmented_gpt import *
import getpass, dotenv, os

dotenv.load_dotenv()
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter your OpenAI API key: ')

## Create a GPT instance that can use tools, and remembers the conversation in a long-term memory storage.

In [4]:
@function
def send_email(
    topic: str = param("The email topic"),
    to: str = param("The recipient email address"),
    content: str = param("The content of the email"),
):
    """Send an email to a recipient"""
    print(f"--- SEND EMAIL ---")
    print(f"TOPIC: {topic}")
    print(f"TO: {to}")
    print(f"CONTENT:\n\n{content}")
    print(f"---")
    return "done."

gpt = AugmentedGPT(functions=[send_email], plugins=[plugins.CalculatorPlugin()], debug=True)

completion = gpt.chat_completion([
    Message(role=Role.USER, content="Send Steve an email and tell him the distance between moon and earth divided by two."),
])

async for response in completion:
    print(response)

[INFO:AugmentedGPT] Register-Function: send_email
[INFO:AugmentedGPT] Register-Function: Calculator-evaluate


[DEBUG:AugmentedGPT] ➡️ Calculator-evaluate: expression=(384400 / 2)


Message(role=<Role.ASSISTANT: 'assistant'>, content=None, name=None, function_call=None, tool_calls=[ToolCall(id='call_nG9nhaaCVAeXltVIsR8SkHhV', function=FunctionCall(name='Calculator-evaluate', arguments={'expression': '(384400 / 2)'}), type='function')], tool_call_id=None)
Message(role=<Role.TOOL: 'tool'>, content='192200.0', name=None, function_call=None, tool_calls=[], tool_call_id='call_nG9nhaaCVAeXltVIsR8SkHhV')


[DEBUG:AugmentedGPT] ➡️ send_email: topic=Distance Between Moon and Earth, to=Steve, content=Hi Steve,

I thought you might be interested to know that the average distance between the Moon and Earth, when divided by two, is approximately 192,200 kilometers.

Best regards.


Message(role=<Role.ASSISTANT: 'assistant'>, content=None, name=None, function_call=None, tool_calls=[ToolCall(id='call_Hvah3GsS0oBw0ZFTCuN1Q2Xk', function=FunctionCall(name='send_email', arguments={'topic': 'Distance Between Moon and Earth', 'to': 'Steve', 'content': 'Hi Steve,\n\nI thought you might be interested to know that the average distance between the Moon and Earth, when divided by two, is approximately 192,200 kilometers.\n\nBest regards.'}), type='function')], tool_call_id=None)
--- SEND EMAIL ---
TOPIC: Distance Between Moon and Earth
TO: Steve
CONTENT:

Hi Steve,

I thought you might be interested to know that the average distance between the Moon and Earth, when divided by two, is approximately 192,200 kilometers.

Best regards.
---
Message(role=<Role.TOOL: 'tool'>, content='done.', name=None, function_call=None, tool_calls=[], tool_call_id='call_Hvah3GsS0oBw0ZFTCuN1Q2Xk')
Message(role=<Role.ASSISTANT: 'assistant'>, content='The email has been successfully sent to Steve, 